In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
#sns.set_palette(palette='deep')
#sns_c = sns.color_palette(palette='deep')
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from imageio import imread
from skimage.transform import resize
import numpy as np
from PIL import Image

In [ ]:
def mass_center(img,is_round=True):
    Y = img.mean(axis=1)
    X = img.mean(axis=0)
    Y_ = np.sum(np.arange(Y.shape[0]) * Y)/np.sum(Y)
    X_ = np.sum(np.arange(X.shape[0]) * X)/np.sum(X)
    if is_round:
        return int(round(X_)),int(round(Y_))
    return X_,Y_

def image_extract(img,newsize):
    x_s = np.where(img.mean(axis=0)!=0)[0].min()
    x_e = np.where(img.mean(axis=0)!=0)[0].max()
    
    y_s = np.where(img.mean(axis=1)!=0)[0].min()
    y_e = np.where(img.mean(axis=1)!=0)[0].max()
    
    x_c,_ = mass_center(img)
    x_s = x_c-newsize[1]//2
    x_e = x_c+newsize[1]//2
    img = img[y_s:y_e,x_s if x_s>0 else 0:x_e if x_e<img.shape[1] else img.shape[1]]
    return np.asarray(Image.fromarray(img).resize(size=newsize).convert('L'))

In [ ]:
from PIL import Image
import imageio

In [ ]:
images = []

for id in range(1, 81):
  if id == 5:
    continue
  
  path = f'/content/drive/MyDrive/CASIA/{str(id).zfill(3)}/nm-01/090/'
  files = os.listdir(path)
  files.sort()
  imgs = [imread(path+f) for f in files]
  imgs = [image_extract(i,(200,150)) for i in imgs]
  imgs = np.minimum(imgs,1)

  for i in range(imgs.shape[0]):
    imgs[i] = np.round(imgs[i])
  
  images.append(imgs)

<ipython-input-6-0cc2b3ab462f>:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imread(path+f) for f in files]


In [ ]:
print(type(images), len(images), [ob.shape for ob in images])

<class 'list'> 79 [(56, 150, 200), (61, 150, 200), (58, 150, 200), (68, 150, 200), (47, 150, 200), (56, 150, 200), (67, 150, 200), (53, 150, 200), (52, 150, 200), (66, 150, 200), (73, 150, 200), (67, 150, 200), (65, 150, 200), (63, 150, 200), (62, 150, 200), (56, 150, 200), (55, 150, 200), (54, 150, 200), (53, 150, 200), (63, 150, 200), (53, 150, 200), (68, 150, 200), (73, 150, 200), (60, 150, 200), (64, 150, 200), (71, 150, 200), (68, 150, 200), (58, 150, 200), (56, 150, 200), (60, 150, 200), (55, 150, 200), (59, 150, 200), (73, 150, 200), (61, 150, 200), (60, 150, 200), (71, 150, 200), (64, 150, 200), (68, 150, 200), (76, 150, 200), (51, 150, 200), (50, 150, 200), (62, 150, 200), (63, 150, 200), (72, 150, 200), (53, 150, 200), (64, 150, 200), (63, 150, 200), (54, 150, 200), (51, 150, 200), (50, 150, 200), (49, 150, 200), (54, 150, 200), (57, 150, 200), (61, 150, 200), (54, 150, 200), (51, 150, 200), (50, 150, 200), (62, 150, 200), (68, 150, 200), (56, 150, 200), (60, 150, 200), (54, 

In [ ]:
global input_time_step
global output_time_step
input_time_step=5
output_time_step=1

In [ ]:
def create_dataset(dataset, time_step=1, output_time_step=5):
  X_for, X_back, Y = [], [], []
  for i in range(time_step, len(dataset)-5):
    a = dataset[(i-time_step):i]
    b = dataset[(i+1):(i+6)][::-1]
    X_for.append(a)
    X_back.append(b)
    Y.append(dataset[i][:])
  return np.array(X_for), np.array(X_back), np.array(Y)

# Loading the models

In [ ]:
import tensorflow as tf

In [ ]:
encoder = tf.keras.models.load_model('/content/drive/MyDrive/enc_casia.h5')
decoder = tf.keras.models.load_model('/content/drive/MyDrive/dec_casia.h5')
lstm_for = tf.keras.models.load_model('/content/drive/MyDrive/lstm_for_casia.h5')
lstm_back = tf.keras.models.load_model('/content/drive/MyDrive/lstm_back_casia.h5')

In [ ]:
merge = tf.keras.models.load_model('/content/drive/MyDrive/merge_casia.h5')

# 10% occlusion

In [ ]:
for i in range(len(images)):
  
  occ_len = int(0.1 * images[i].shape[0])
  occ_len1 = occ_len // 2
  occ_len2 = occ_len - 1

  x = images[i]
  enc_x = encoder.predict(np.expand_dims(x, axis=3))

  out_f1 = []
  out_f2 = []
  out_b1 = []
  out_b2 = []

  xf = enc_x[:5]
  xb = enc_x[5 + occ_len1 : 10 + occ_len1][::-1]

  for j in range(occ_len1):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f1.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len1):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b1.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b1 = out_b1[::-1]

  out_f1 = np.array(out_f1)
  out_b1 = np.array(out_b1)

  d_f1 = np.round(decoder.predict(out_f1.reshape(occ_len1, 38, 50, 8)))
  d_b1 = np.round(decoder.predict(out_b1.reshape(occ_len1, 38, 50, 8)))

  d1 = np.append(d_f1, d_b1, axis=3)

  f1 = merge.predict(d1)

  temp = images[i].shape[0]

  xf = enc_x[temp - occ_len2 - (2 * 5) : temp - occ_len2 - 5]
  xb = enc_x[temp - 5 :][::-1]

  for j in range(occ_len2):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f2.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len2):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b2.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b2 = out_b2[::-1]

  out_f2 = np.array(out_f2)
  out_b2 = np.array(out_b2)

  d_f2 = np.round(decoder.predict(out_f2.reshape(occ_len2, 38, 50, 8)))
  d_b2 = np.round(decoder.predict(out_b2.reshape(occ_len2, 38, 50, 8)))

  d2 = np.append(d_f2, d_b2, axis=3)

  f2 = merge.predict(d2)

  temp1 = np.vstack((np.array(x[:5]), np.round(f1.reshape(occ_len1, 150, 200)), np.array(x[5 + occ_len1:temp - occ_len2 - 5]), np.round(f2.reshape(occ_len2, 150, 200)), np.array(x[temp - 5:])))

  assert temp1.shape[0] == temp

  k = temp // 20

  for inner_i in range(k):
    j = 20*inner_i
    gei = np.mean(temp1[j:j+20], axis=0)
    im = Image.fromarray(gei)
    save_path = f'/content/drive/MyDrive/GEI_10/{str(i).zfill(3)}'
    if not os.path.exists(save_path):
      os.makedirs(save_path)
    imageio.imwrite(save_path+'/'+str(inner_i).zfill(3)+'.png', gei)

1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 67ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 28ms/step


...

1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 21ms/step


# 20% occlusion

In [ ]:
for i in range(len(images)):
  
  occ_len = int(0.2 * images[i].shape[0])
  occ_len1 = occ_len // 2
  occ_len2 = occ_len - 1

  x = images[i]
  enc_x = encoder.predict(np.expand_dims(x, axis=3))

  out_f1 = []
  out_f2 = []
  out_b1 = []
  out_b2 = []

  xf = enc_x[:5]
  xb = enc_x[5 + occ_len1 : 10 + occ_len1][::-1]

  for j in range(occ_len1):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f1.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len1):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b1.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b1 = out_b1[::-1]

  out_f1 = np.array(out_f1)
  out_b1 = np.array(out_b1)

  d_f1 = np.round(decoder.predict(out_f1.reshape(occ_len1, 38, 50, 8)))
  d_b1 = np.round(decoder.predict(out_b1.reshape(occ_len1, 38, 50, 8)))

  d1 = np.append(d_f1, d_b1, axis=3)

  f1 = merge.predict(d1)

  temp = images[i].shape[0]

  xf = enc_x[temp - occ_len2 - (2 * 5) : temp - occ_len2 - 5]
  xb = enc_x[temp - 5 :][::-1]

  for j in range(occ_len2):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f2.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len2):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b2.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b2 = out_b2[::-1]

  out_f2 = np.array(out_f2)
  out_b2 = np.array(out_b2)

  d_f2 = np.round(decoder.predict(out_f2.reshape(occ_len2, 38, 50, 8)))
  d_b2 = np.round(decoder.predict(out_b2.reshape(occ_len2, 38, 50, 8)))

  d2 = np.append(d_f2, d_b2, axis=3)

  f2 = merge.predict(d2)

  temp1 = np.vstack((np.array(x[:5]), np.round(f1.reshape(occ_len1, 150, 200)), np.array(x[5 + occ_len1:temp - occ_len2 - 5]), np.round(f2.reshape(occ_len2, 150, 200)), np.array(x[temp - 5:])))

  assert temp1.shape[0] == temp

  k = temp // 20

  for inner_i in range(k):
    j = 20*inner_i
    gei = np.mean(temp1[j:j+20], axis=0)
    im = Image.fromarray(gei)
    save_path = f'/content/drive/MyDrive/GEI_20/{str(i).zfill(3)}'
    if not os.path.exists(save_path):
      os.makedirs(save_path)
    imageio.imwrite(save_path+'/'+str(inner_i).zfill(3)+'.png', gei)

1/1 [==============================] - 0s 82ms/step


1/1 [==============================] - 0s 76ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 67ms/step


...

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 22ms/step


# 30% occlusion

In [ ]:
for i in range(len(images)):
  
  occ_len = int(0.3 * images[i].shape[0])
  occ_len1 = occ_len // 2
  occ_len2 = occ_len - 1

  x = images[i]
  enc_x = encoder.predict(np.expand_dims(x, axis=3))

  out_f1 = []
  out_f2 = []
  out_b1 = []
  out_b2 = []

  xf = enc_x[:5]
  xb = enc_x[5 + occ_len1 : 10 + occ_len1][::-1]

  for j in range(occ_len1):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f1.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len1):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b1.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b1 = out_b1[::-1]

  out_f1 = np.array(out_f1)
  out_b1 = np.array(out_b1)

  d_f1 = np.round(decoder.predict(out_f1.reshape(occ_len1, 38, 50, 8)))
  d_b1 = np.round(decoder.predict(out_b1.reshape(occ_len1, 38, 50, 8)))

  d1 = np.append(d_f1, d_b1, axis=3)

  f1 = merge.predict(d1)

  temp = images[i].shape[0]

  xf = enc_x[temp - occ_len2 - (2 * 5) : temp - occ_len2 - 5]
  xb = enc_x[temp - 5 :][::-1]

  for j in range(occ_len2):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f2.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len2):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b2.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b2 = out_b2[::-1]

  out_f2 = np.array(out_f2)
  out_b2 = np.array(out_b2)

  d_f2 = np.round(decoder.predict(out_f2.reshape(occ_len2, 38, 50, 8)))
  d_b2 = np.round(decoder.predict(out_b2.reshape(occ_len2, 38, 50, 8)))

  d2 = np.append(d_f2, d_b2, axis=3)

  f2 = merge.predict(d2)

  temp1 = np.vstack((np.array(x[:5]), np.round(f1.reshape(occ_len1, 150, 200)), np.array(x[5 + occ_len1:temp - occ_len2 - 5]), np.round(f2.reshape(occ_len2, 150, 200)), np.array(x[temp - 5:])))

  assert temp1.shape[0] == temp

  k = temp // 20

  for inner_i in range(k):
    j = 20*inner_i
    gei = np.mean(temp1[j:j+20], axis=0)
    im = Image.fromarray(gei)
    save_path = f'/content/drive/MyDrive/GEI_30/{str(i).zfill(3)}'
    if not os.path.exists(save_path):
      os.makedirs(save_path)
    imageio.imwrite(save_path+'/'+str(inner_i).zfill(3)+'.png', gei)

1/1 [==============================] - 0s 73ms/step


1/1 [==============================] - 0s 79ms/step


1/1 [==============================] - 0s 92ms/step


1/1 [==============================] - 0s 78ms/step


1/1 [==============================] - 0s 22ms/step


...

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 20ms/step


# 40% occlusion

In [ ]:
for i in range(len(images)):
  
  occ_len = int(0.4 * images[i].shape[0])
  occ_len1 = occ_len // 2
  occ_len2 = occ_len - 1

  x = images[i]
  enc_x = encoder.predict(np.expand_dims(x, axis=3))

  out_f1 = []
  out_f2 = []
  out_b1 = []
  out_b2 = []

  xf = enc_x[:5]
  xb = enc_x[5 + occ_len1 : 10 + occ_len1][::-1]

  for j in range(occ_len1):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f1.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len1):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b1.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b1 = out_b1[::-1]

  out_f1 = np.array(out_f1)
  out_b1 = np.array(out_b1)

  d_f1 = np.round(decoder.predict(out_f1.reshape(occ_len1, 38, 50, 8)))
  d_b1 = np.round(decoder.predict(out_b1.reshape(occ_len1, 38, 50, 8)))

  d1 = np.append(d_f1, d_b1, axis=3)

  f1 = merge.predict(d1)

  temp = images[i].shape[0]

  xf = enc_x[temp - occ_len2 - (2 * 5) : temp - occ_len2 - 5]
  xb = enc_x[temp - 5 :][::-1]

  for j in range(occ_len2):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f2.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len2):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b2.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b2 = out_b2[::-1]

  out_f2 = np.array(out_f2)
  out_b2 = np.array(out_b2)

  d_f2 = np.round(decoder.predict(out_f2.reshape(occ_len2, 38, 50, 8)))
  d_b2 = np.round(decoder.predict(out_b2.reshape(occ_len2, 38, 50, 8)))

  d2 = np.append(d_f2, d_b2, axis=3)

  f2 = merge.predict(d2)

  temp1 = np.vstack((np.array(x[:5]), np.round(f1.reshape(occ_len1, 150, 200)), np.array(x[5 + occ_len1:temp - occ_len2 - 5]), np.round(f2.reshape(occ_len2, 150, 200)), np.array(x[temp - 5:])))

  print(temp1.shape[0], temp)
  assert temp1.shape[0] == temp

  k = temp // 20

  for inner_i in range(k):
    j = 20*inner_i
    gei = np.mean(temp1[j:j+20], axis=0)
    im = Image.fromarray(gei)
    save_path = f'/content/drive/MyDrive/GEI_40/{str(i).zfill(3)}'
    if not os.path.exists(save_path):
      os.makedirs(save_path)
    imageio.imwrite(save_path+'/'+str(inner_i).zfill(3)+'.png', gei)

1/1 [==============================] - 0s 30ms/step
56 56


1/1 [==============================] - 0s 23ms/step


61 61
1/1 [==============================] - 0s 27ms/step


58 58
1/1 [==============================] - 0s 24ms/step


68 68
1/1 [==============================] - 0s 22ms/step
47 47


...

59 59


1/1 [==============================] - 0s 34ms/step


62 62


1/1 [==============================] - 0s 20ms/step


63 63


# 50% occlusion

In [ ]:
for i in range(len(images)):
  
  occ_len = int(0.5 * images[i].shape[0])
  occ_len1 = occ_len // 2
  occ_len2 = occ_len - 1

  x = images[i]
  enc_x = encoder.predict(np.expand_dims(x, axis=3))

  out_f1 = []
  out_f2 = []
  out_b1 = []
  out_b2 = []

  xf = enc_x[:5]
  xb = enc_x[5 + occ_len1 : 10 + occ_len1][::-1]

  for j in range(occ_len1):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f1.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len1):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b1.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b1 = out_b1[::-1]

  out_f1 = np.array(out_f1)
  out_b1 = np.array(out_b1)

  d_f1 = np.round(decoder.predict(out_f1.reshape(occ_len1, 38, 50, 8)))
  d_b1 = np.round(decoder.predict(out_b1.reshape(occ_len1, 38, 50, 8)))

  d1 = np.append(d_f1, d_b1, axis=3)

  f1 = merge.predict(d1)

  temp = images[i].shape[0]

  xf = enc_x[temp - occ_len2 - (2 * 5) : temp - occ_len2 - 5]
  xb = enc_x[temp - 5 :][::-1]

  for j in range(occ_len2):
    xf = xf.reshape(1, 5, -1)
    l = lstm_for.predict(xf)
    out_f2.append(l[0])
    xf = np.vstack((xf[0][1:], l))

  for j in range(occ_len2):
    xb = xb.reshape(1, 5, -1)
    l = lstm_back.predict(xb)
    out_b2.append(l[0])
    xb = np.vstack((xb[0][1:], l))
  
  out_b2 = out_b2[::-1]

  out_f2 = np.array(out_f2)
  out_b2 = np.array(out_b2)

  d_f2 = np.round(decoder.predict(out_f2.reshape(occ_len2, 38, 50, 8)))
  d_b2 = np.round(decoder.predict(out_b2.reshape(occ_len2, 38, 50, 8)))

  d2 = np.append(d_f2, d_b2, axis=3)

  f2 = merge.predict(d2)

  temp1 = np.vstack((np.array(x[:5]), np.round(f1.reshape(occ_len1, 150, 200)), np.array(x[5 + occ_len1:temp - occ_len2 - 5]), np.round(f2.reshape(occ_len2, 150, 200)), np.array(x[temp - 5:])))

  print(temp1.shape[0], temp)
  assert temp1.shape[0] == temp

  k = temp // 20

  for inner_i in range(k):
    j = 20*inner_i
    gei = np.mean(temp1[j:j+20], axis=0)
    im = Image.fromarray(gei)
    save_path = f'/content/drive/MyDrive/GEI_50/{str(i).zfill(3)}'
    if not os.path.exists(save_path):
      os.makedirs(save_path)
    imageio.imwrite(save_path+'/'+str(inner_i).zfill(3)+'.png', gei)

1/1 [==============================] - 0s 21ms/step
56 56


1/1 [==============================] - 0s 38ms/step


61 61


1/1 [==============================] - 0s 35ms/step


58 58


...

1/1 [==============================] - 0s 23ms/step


59 59
1/1 [==============================] - 0s 28ms/step


62 62


1/1 [==============================] - 0s 36ms/step


63 63
